Due to a fault with Google Colab, every time you open this notebook, you need to run the block below, then restart the environment. Then you can continue as normal.

Jupyter Notebook should not have this problem, and can be run as normal.

In [ ]:
!pip uninstall -y tweepy

!pip install tweepy

Found existing installation: tweepy 3.10.0
Uninstalling tweepy-3.10.0:
  Successfully uninstalled tweepy-3.10.0
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## Set API Keys for Tweepy

In [ ]:
import tweepy
import pandas as pd

auth = tweepy.OAuth1UserHandler('CHmY1hL3cbijd9xK7LbHpJIu0', 'LHrXZY2Kfje1XpncnykLV0ggCze9szBfzsVo4EnZK0qpivckG8')
auth.set_access_token('859520503610040323-vPifjf4ONZFnfkVZZGzjeZWYKu3Wubz', 'wQDWZ18OdO3sMq4w6tAkKPPjFYQerDAKGddp8UosUzzQp')

api = tweepy.API(auth, wait_on_rate_limit=True)

## Get Primary Tweets

"fromsoftware": Results of the search "fromsoftware". ('fromsoftware') \[2022-03-07 17:25 UNTIL 2022-03-20 16:01\] : 13,127 Tweets

"fromsoftware_question": The above query, but with the '?' tag, to filter for questions ('fromsoftware %3F') \[2022-03-13 10:58:14 UNTIL 2022-03-21 18:04:12\] : 808 Tweets 

"at_fromsoftware": Posts containing specific mentions of the fromsoftware_pr account. ('%40Fromsoftware_pr') \[2022-03-13 10:42:17 UNTIL 2022-03-21 18:10:02\] : 1849 Tweets

"at_fromsoftware_question": The above query, but with the '?' tag, to filter for questions ('%40Fromsoftware_pr %3F') \[2022-03-13 12:32:48 UNTIL 2022-03-20 16:01\] : 279 Tweets

"current_set": The aggregation of the above.

**Note: Recommended not to run! Later sections can import local datasets.**

In [ ]:
# Collect tweets

query = "fromsoftware" + " -filter:retweets"
date_before = "2022-03-20"

tweets = tweepy.Cursor(api.search_tweets, q=query, lang="en").items(100000)
tweets_list = [[tweet.created_at, tweet.user.screen_name, tweet.user.location, tweet.text] for tweet in tweets]
tweets_df = pd.DataFrame(data=tweets_list, columns=['date', 'user', 'location', 'text'])

# A good idea to save downloaded tweets as CSV
tweets_df.to_csv ('current_set_' + '.csv', quotechar='"', encoding='utf8', index = False, header=True)
date_before = str(tweets_df.iloc[len(tweets_df) - 1, 0]).split()[0]
print("Done downloading data " + ". Now find tweets from" + date_before)

KeyboardInterrupt: ignored

In [ ]:
# Collect tweets
query = "%40Fromsoftware_pr" + " -filter:retweets"
date_before = "2022-03-20"

tweets = tweepy.Cursor(api.search_tweets, q=query, lang="en").items(100000)
tweets_list = [[tweet.created_at, tweet.user.screen_name, tweet.user.location, tweet.text] for tweet in tweets]
tweets_df = pd.DataFrame(data=tweets_list, columns=['date', 'user', 'location', 'text'])

# A good idea to save downloaded tweets as CSV
tweets_df.to_csv ('new_set_' + '.csv', quotechar='"', encoding='utf8', index = False, header=True)
date_before = str(tweets_df.iloc[len(tweets_df) - 1, 0]).split()[0]
print("Done downloading data " + ". Now find tweets from" + date_before)

Rate limit reached. Sleeping for: 234


Done downloading data . Now find tweets from2022-03-13


## Preprocessing

Global preprocessing of the dataset. Removal of punctuation, line breaks, lowercasing, embedded twitter links, mentions, hashtag characters, and duplicates.

In [ ]:
# Load the regular expression library
import re

# Uncomment the line below if loading from previously saved CSV
tweets_df = pd.read_csv('current_set.csv', quotechar='"', encoding='utf8') #'Remove punctuation' error means file not found

# Remove punctuation
tweets_df['text_processed'] = tweets_df['text'].map(lambda x: re.sub('[,\\.!?]', ' ', x))

# Remove unnecessary line breaks
tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: re.sub(r"\n", '', x))

# Convert the titles to lowercase
tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: x.lower())

# Get rid of https://t, fragments ("co/")
tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: re.sub(r'http\S+', '', x))
tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: re.sub(r'co/\S+', '', x))

#Get rid of single letters
#tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: re.sub(r' . ', ' ', x))

#Get rid of mentions "@..."a
tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: re.sub(r'@\S+', '', x))

#Remove hashtags
tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: re.sub('#', '', x))

# Print out the first rows 
print(tweets_df['text_processed'].head())

# Removing duplicate tweets
unique_tweets = list(dict.fromkeys(list(tweets_df['text_processed'].values)))
unique_tweets = [t for t in unique_tweets if t]

#tweets_df = pd.read_csv('current_set.csv', quotechar='"', encoding='utf8')

#Write to File (to allow referencing)
with open('current_set_clean.csv', 'w') as output:

    for tweet in unique_tweets:    
        output.write('{0}\n'.format(tweet))   

output.close()

print(len(unique_tweets))

FileNotFoundError: ignored

##  **Question 1: What are the most common complaints and criticisms for FromSoftware games?**

**Q1: Sentiment Analysis**

This section takes the tweets, runs them through a sentiment analysis algorithm, then

In [ ]:
def SentimentAnalysis(data):

  

## **Q1: Sentiment Analysis**

Here we are using sentiment analysis to understand if a tweet is a complaint or critism. For this, we will use Vader, since other 

In [ ]:
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 6.8 MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

sentiments = []

for tweet_text in unique_tweets:
    sentiments.append([analyzer.polarity_scores(tweet_text)['compound'], tweet_text])

sentiments.sort()

print("Negativity \t Tweet")
negative_tweets = []

for score in sentiments:
    if(score[0] < -0.5):
      print('{0} \t {1}\n'.format(score[0], score[1]))
      negative_tweets.append(score)

Negativity 	 Tweet
-0.9873 	      oh my god     🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥

-0.9843 	 oh  f&amp;*k  a giant lobster   aw hell no  why  fromsoftware  why  😭😭😭😭😭😭😭😭  

-0.9661 	 //i hate when i get stuck in a fromsoftware gameelden ring beating my ass 😭😭😭

-0.9652 	 i wanna start elden ring so bad man but i’ve got so much other shit to do 😭bungie and fromsoftware fucked me up dr…  

-0.9623 	 who the fuck designed this cancer cancer fight  the red bitch in the swamp with the knights that keep spawning fuck…  

-0.9557 	 🔴 bruh im scared of giant mutant hands | playing elden ring and i'm really not happy about it   😱😱😱…  

-0.9485 	  wtf were you thinking with the fire giant  this shit is dumb as fuck  im level 98 and this mf take…  

-0.9428 	  i understand making a challenge  but ffs  fix your broken mechanics  why are you instant kills aoe…  

-0.9423 	  i killed that fucker as soon as he said i was maidenless  though it was tough as hell considering i just s…  

-0.9325 	  fromsoftware you ev

## **Question 2: Who are the most noteworthy boss enemies in FromSoftware games? Why?**

#Q2.1: Custom Entity Recognition (Rule-based entity recognition)

This code takes the 9 manually-constructed entity listings, and aggregates them into a single 'knowledge base' file, to be loaded into the nlp pipeline prior to named entity recognition as rules. This will allow greater accuracy during NER, especially considering abnormal phrase usage (as is common from both a social media corpus, and the video game domain):

*videogame_fromsoftware.jsonl* **Video Games created by FromSoftware**

*person_fromsoftware.jsonl* **Individuals related to FromSoftware.**

*org_fromsoftware* **Corporate entities related to FromSoftware.**

*kb/er_bosses.jsonl* **Boss-class enemies in Elden Ring**

*kb/sk_bosses.jsonl* **Boss-class enemies in Sekiro: Shadows Die Twice**

*kb/bb_bosses.jsonl* **Boss-class enemies in Bloodborne**

*kb/ds3_bosses.jsonl* **Boss-class enemies in Dark Souls 3 (+ DLC)**

*kb/ds2_bosses.jsonl* **Boss-class enemies in Dark Souls 2 (+ DLC)**

*kb/ds_bosses.jsonl* **Boss-class enemies in Dark Souls**

In [ ]:
#Prepare dataset
!cat kb/*.jsonl > fromsoftware.jsonl # Aggregate all files in the 'kb' dir

In [ ]:
#Install
!pip install spacy-transformers
!pip install spacy-entity-linker
!python3 -m spacy download en_core_web_md

In [ ]:
#Import
import numpy as np
import spacy
from spacy.pipeline import EntityRuler
import en_core_web_md
from spacy.language import Language

from spacy.symbols import ORTH

In [ ]:
#Create Pipeline
nlp = spacy.load('en_core_web_md')

ruler = nlp.add_pipe('entity_ruler', before='ner', config={"validate": True})
ruler.from_disk("./fromsoftware.jsonl")

boss_labels = [label for label in ruler.labels if 'BOSSES' in label]

print(boss_labels)

nlp.to_disk("./rule-based-model")

     |████████████████████████████████| 45.7 MB 1.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


FileNotFoundError: ignored

## **Q2.2: Entity Dict Construction**

Here, the corpus is fed into the NLP pipeline, which uses the rule-based KB lookup prior to pre-trained NER processes. Mentions of entities are identified, and added to a dictionary, with their count incremented in each subsequent mention.

**Note: Q2.2 was not used directly in solving Question 2, but allowed iterative development of the knowledge base, by identifying remaining aliases and terms. Code left for insight.**

In [ ]:
ents = {}

for tweet_text in unique_tweets:
    
    doc = nlp(tweet_text)   

    for ne in doc.ents:
        
        text_ = tweet_text[ne.start_char:ne.end_char]
        pkey = ne.ent_id_ if ne.ent_id_ else text_ #If we provide an ID, use it to disambiguate.

        #Give dict entry if label type is new
        if ne.label_ not in ents:
            ents[ne.label_] = []
            ents[ne.label_].append((pkey, text_, 0))

        #Give text entity an entry in dict's label if new
        index = [ents[ne.label_].index(item) for item in ents[ne.label_] if item[0] == pkey]  
        if len(index) == 0:        
            ents[ne.label_].append((pkey, text_, 1))
            #print('\tMentions:{0} {1} : {2} [{3}].'.format(1, ne.label_, text_, pkey))

        #Add count to already identified entry
        else:
            count = ents[ne.label_][index[0]][2] +1
            ents[ne.label_][index[0]] = (pkey, text_, count)
            #print('\tMentions:{0} {1} : {2} [{3}].'.format(count, ne.label_, text_, pkey))

## **Q2.2: Output Entity Dictionary**

For closer examination of identified entities, and their counts, this code can be ran to output tokens, stratified by category, to a local file. Single-use entities are omitted.

This data was used in a interative process; the dictionaries were updated with unidentified tokens identified in this file, and the process was re-run with the existing dictionary, until no new significant and likely aliases/variations on tokens of interest occured.

In [ ]:
with open('ents.csv', 'w') as output:

    for ent in ents.items():

        ent[1].sort(key=lambda tup: tup[2], reverse=True)    
        output.write('{0}\n'.format(ent[0]))  
        
        for txt in ent[1]:
            line = '{0},{1}\n'.format(txt[2], txt[1])
            if(int(txt[2]) > 1):
                output.write(line)  

output.close()

## **Q2.3: Token-based Sentiment Aggregation**

VaderSentiment, an external sentiment-analysis package, is loaded, and used to label each 'tweet' with an estimated sentiment, in the range of -1 (Highly Negative), to 1 (Highly Positive).

Next, a dictionary of entities (by their label) is constructed, and the corpus is iterated over. A similar process to earlier is performed, where entities are identified and added to the dictionary, but the sentiment of their use context is aggregated, too. This allows a rudimentary understanding of the public's sentiment toward sufficiently popular tokens.

In [ ]:
#Install
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 16.0 MB/s 


In [ ]:
#Import
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
#Use VaderSentiment to obtain sentiment values for each tweet.
analyzer = SentimentIntensityAnalyzer()

#Create a list of tweets with sentiments attached.
scores = []
for tweet_text in unique_tweets:
    scores.append([analyzer.polarity_scores(tweet_text)['compound'], tweet_text])

In [ ]:
#Appends aggregate sentiment to each entity
aggents = {} #Dictionary(Labels) of Lists(Entities) of Tuples(Text, Count, etc...)
for idx, scored_tweet in enumerate(scores):
     
    #Get sentiment of parent tweet.
    current = scores.index(scored_tweet)
    tweet_sentiment = scored_tweet[0] 
    tweet_text = scored_tweet[1]

    if(current % 100 == 0):
        print('Tweet {0} of {1}'.format(current, len(scores)))

    doc = nlp(tweet_text)   
    for ne in doc.ents:
        
        text_ = tweet_text[ne.start_char:ne.end_char]
        pkey = ne.ent_id_ if ne.ent_id_ else text_ #Use ID of entity (not raw text) as primary key
        
        #Create new label category (i.e. ORG, NORD, ...)
        if ne.label_ not in aggents:
            aggents[ne.label_] = []
            #aggents[ne.label_].append((pkey, text_, 1, 0, [idx]))
        
        #Create new entry for entity if dict[label] does not contain it
        index = [aggents[ne.label_].index(item) for item in aggents[ne.label_] if item[0] == pkey]  
        if len(index) == 0:        
            aggents[ne.label_].append((pkey, text_, 1, tweet_sentiment, [(current, ne.start_char, ne.end_char)]))
            #print('\tMentions:{0} Opinion / weighted: [{1}/{2}] {3} : {4}.'.format(1, tweet_sentiment, tweet_sentiment, ne.label_, text_))
        
        #Amend values of existing entry if dict[label] contains it
        else:
            count = aggents[ne.label_][index[0]][2] +1 #Incremented frequency
            aggregate_sentiment = aggents[ne.label_][index[0]][3] + tweet_sentiment #Running sentiment
            tweet_sources = aggents[ne.label_][index[0]][4] + [(current, ne.start_char, ne.end_char)] #Annotation ref to referencing tweet
            aggents[ne.label_][index[0]] = (pkey, text_, count, aggregate_sentiment, tweet_sources)
            #print('\tMentions:{0} Opinion / weighted: [{1}/{2}] {3} : {4}.'.format(count, aggregate_sentiment, aggregate_sentiment/count, ne.label_, text_))
        
        #print('\tNE found: ', ne.start_char, ne.end_char, ne.label_, tweet_text[ne.start_char:ne.end_char])

Tweet 0 of 12889
Tweet 100 of 12889
Tweet 200 of 12889
Tweet 300 of 12889
Tweet 400 of 12889
Tweet 500 of 12889
Tweet 600 of 12889
Tweet 700 of 12889
Tweet 800 of 12889
Tweet 900 of 12889
Tweet 1000 of 12889
Tweet 1100 of 12889
Tweet 1200 of 12889
Tweet 1300 of 12889
Tweet 1400 of 12889
Tweet 1500 of 12889
Tweet 1600 of 12889
Tweet 1700 of 12889
Tweet 1800 of 12889
Tweet 1900 of 12889
Tweet 2000 of 12889
Tweet 2100 of 12889
Tweet 2200 of 12889
Tweet 2300 of 12889
Tweet 2400 of 12889
Tweet 2500 of 12889
Tweet 2600 of 12889
Tweet 2700 of 12889
Tweet 2800 of 12889
Tweet 2900 of 12889
Tweet 3000 of 12889
Tweet 3100 of 12889
Tweet 3200 of 12889
Tweet 3300 of 12889
Tweet 3400 of 12889
Tweet 3500 of 12889
Tweet 3600 of 12889
Tweet 3700 of 12889
Tweet 3800 of 12889
Tweet 3900 of 12889
Tweet 4000 of 12889
Tweet 4100 of 12889
Tweet 4200 of 12889
Tweet 4300 of 12889
Tweet 4400 of 12889
Tweet 4500 of 12889
Tweet 4600 of 12889
Tweet 4700 of 12889
Tweet 4800 of 12889
Tweet 4900 of 12889
Tweet 5000 o

## **Q2.3: Output Entity Dictionary**

This code can be optionally ran to examine the results of aggregated entity collection.

Format: # Mentions, Total Weight, Averaged Weight, Token ID

In [ ]:
#Save ents to file
with open('aggents.csv', 'w') as output:
    for ent in aggents.items():
        ent[1].sort(key=lambda tup: tup[3], reverse=True)
        output.write('Count, Sentiment, Weighted Sentiment, Token, Tweets Mentioned\n')
        output.write('Category: ' + ent[0] + '\n')  
        for txt in ent[1]:
            line = '{0},{1},{2},{3}, [{4}]\n'.format(txt[2], txt[3], txt[3]/txt[2], txt[1], txt[4])
            output.write(line)  
output.close()

## Q2.4: Keyword Extraction

Using nltk, entities are collected into three categories, based on frequency: recognised (n>=10), notable (n>=30), eminent (n>=70).


In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from collections import Counter

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

pd.set_option('display.max_rows', None)

In [ ]:
#Obtain entities of interest
entities_recognised, entities_notable, entities_eminent = list(), list(), list()
threshold_recognised, threshold_notable, threshold_eminent = 10, 30, 70

for label in boss_labels: #Created in 2.1
    for ent in aggents[label]:
        frequency = ent[2]
        
        if(frequency > threshold_eminent):
            entities_eminent.append((label, ent))
            continue
        
        if(frequency > threshold_notable):
            entities_notable.append((label, ent))
            continue
            
        if(frequency > threshold_recognised):
            entities_recognised.append((label, ent))

#Print each entity, including frequency (f) and (weighted) sentiment (s)  
print('\nEminent Entities:')
for entity in entities_eminent:
    print(' {0} (f:{1}) (s:{2}),'.format(entity[1][1], entity[1][2], round(entity[1][3]/entity[1][2], 2)), end='')  
   
print('\nNotable Entities:')
for entity in entities_notable:
    print('{0} (f:{1}) (s:{2})'.format(entity[1][1], entity[1][2], round(entity[1][3]/entity[1][2], 2)), end='')  

print('\nRecognised Entities:')
for entity in entities_recognised:
    print(' {0} (f:{1}) (s:{2})'.format(entity[1][1], entity[1][2], round(entity[1][3]/entity[1][2], 2)), end='')  
    
        
#For each entity, obtain key information
def get_key_information(entities_list):
    for entity in entities_list:

        label_of_interest = entity[0]
        token_of_interest = entity[1][0]
        plaintext_name = entity[1][1]

        #Obtain a list of tweet indexes which refer to the token of interest
        index = [aggents[label_of_interest].index(item) for item in aggents[label_of_interest] if item[0] == token_of_interest] 

        #Lookup token location info ([4]) and store locally in tokenlist
        tokens = list()
        tokenlist = list(aggents[label_of_interest][index[0]][4]) #All tweets, even duplicates

        unique_tweets_copy = unique_tweets.copy()

        #For all token instances, replace with single-word lemma of token 
        for idx, tweet_annotation in enumerate(tokenlist):
            base_string = unique_tweets[tweet_annotation[0]]     
            srt, end = tweet_annotation[1], tweet_annotation[2]

            lemma_string = base_string[:srt] + plaintext_name + ' ' + base_string[end + 1:] #Convert token instances to base form

            unique_tweets_copy[tweet_annotation[0]] = lemma_string 

        tokenidxs = list(dict((idx, (srt, end)) for idx, srt, end in tokenlist)) #Now words are replaced, only care about unique tweets

        for idx in tokenidxs:
            tokens = tokens + unique_tweets_copy[idx].split() #Add tokens of unique tweet to list for 'Grams

        #Extract non-trivial tokens
        is_label = lambda pos: pos[:2] in ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'RB', 'RBR', 'RBS', 'VB', 'VBG', 'VBN', 'VBZ')
        tokenized = nltk.word_tokenize(' '.join(tokens))
        token_label_subset = [word for (word, pos) in nltk.pos_tag(tokenized) if is_label(pos)] 

        #Create ascending list based on frequency, omitting single mentions
        x = Counter(token_label_subset)
        x = Counter({k: c for k, c in x.items() if c >= 2})

        entities_list[entities_list.index(entity)] = entity , tuple(x.most_common()), tuple(tokens)

    return entities_list

#Create keyword lists for eminent and notable boss entities
entities_eminent = get_key_information(entities_eminent)
entities_notable = get_key_information(entities_notable)
    
for entity in entities_eminent:
    display(entity[1])
    
for entity in entities_notable:
    display(entity[1])

,bigram,freq
35,"(starscourge, radahn)",16
110,"(general, radahn)",16
41,"(elden, ring)",14
249,"(radahn, in)",12
44,"(radahn, is)",10
215,"(of, the)",10
45,"(is, the)",9
214,"(one, of)",8
152,"(eldenring, fromsoftware)",8
11,"(fromsoftware, boss)",6


,trigram,freq
267,"(general, radahn, in)",7
229,"(one, of, the)",6
553,"(about, starscourge, radahn)",5
554,"(starscourge, radahn, elden)",5
548,"(excuse, me, while)",5
549,"(me, while, i)",5
550,"(while, i, gush)",5
551,"(i, gush, about)",5
552,"(gush, about, starscourge)",5
555,"(radahn, elden, ring’s)",5


## Q2.4: Wordcloud visualition

Optionally generate wordclouds for the key words gathered for each of the eminent and notable entities. This may allow the user a more holistic understanding of the public sentiment toward each.

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordclouds = list()

def get_wordcloud(entities):
    
    wordclouds_local = list()
    
    for entity in entities:
        # join the words of the different tweets together into one string
        new_long_string = ' '.join(list(entity[2]))

        # create a WordCloud object
        wordcloud = WordCloud(background_color="white", max_words=1000, contour_width=3, contour_color='steelblue', width=1920, height=1080)

        # generate a word cloud
        wordcloud.generate(new_long_string)
        wordclouds_local.append(wordcloud)
    
    return wordclouds_local
 
wordclouds += get_wordcloud(entities_eminent)
wordclouds += get_wordcloud(entities_notable)

print(wordclouds)

## Question 3